In [1]:
import numpy as np
from astropy.units import Quantity

from rls.simulation import Simulation
from rls.data_types import Particle, SimQuantity
from rls.formula.physics import lorentz_factor
from rls.formula.conversions import energy_to_speed
from rls.models import WhistlerAtGradientModel

sim = Simulation(model=WhistlerAtGradientModel(
    Bw=0.05, w_wce=0.05, sw=1.75, Bh=-0.8, B0=-1.0, theta=0.0,
))
model = sim.model
units = model.units
V_factor = (units.L_factor / units.T_factor)
W_factor = units.W_factor

W_para_min = Quantity(1, "eV")
W_para_min = SimQuantity(W_factor.user_to_code(W_para_min), W_para_min)
V_para_min = energy_to_speed(W_para_min, units)

W_para_max = Quantity(1000, "eV")
W_para_max = SimQuantity(W_factor.user_to_code(W_para_max), W_para_max)
V_para_max = energy_to_speed(W_para_max, units)

W_perp = Quantity(1, "eV")
W_perp = SimQuantity(W_factor.user_to_code(W_perp), W_perp)
V_perp = energy_to_speed(W_perp, units)

c = units.c.code
R = model.R.code
Tc = units.electron.Tc(model.B0).code
uz0 = np.linspace(V_para_min.code, V_para_max.code, 25)
ux0 = np.zeros_like(uz0) + V_perp.code
uy0 = np.zeros_like(uz0)
x0 = np.zeros_like(uz0)
y0 = np.zeros_like(uz0)
z0 = np.zeros_like(uz0) - 3.0 * R
g0 = lorentz_factor(ux0, uy0, uz0, c)
ICs = Particle(units.electron, 0.0, g0, x0, y0, z0, ux0, uy0, uz0)
sim.run(
    initial_conditions=ICs,
    run_time=2000.0 * Tc,
    step_size=1e-2 * Tc,
    save_intervals=-1,
    log_intervals=10,
)

Pushed 0/200000 steps (0.00%, 0.000000 ms/step, estimated remaining run time = 0.00 min)
Pushed 20000/200000 steps (10.00%, 0.000022 ms/step, estimated remaining run time = 0.07 min)
Pushed 40000/200000 steps (20.00%, 0.000022 ms/step, estimated remaining run time = 0.06 min)
Pushed 60000/200000 steps (30.00%, 0.000022 ms/step, estimated remaining run time = 0.05 min)
Pushed 80000/200000 steps (40.00%, 0.000020 ms/step, estimated remaining run time = 0.04 min)
Pushed 100000/200000 steps (50.00%, 0.000020 ms/step, estimated remaining run time = 0.03 min)
Pushed 120000/200000 steps (60.00%, 0.000023 ms/step, estimated remaining run time = 0.03 min)
Pushed 140000/200000 steps (70.00%, 0.000024 ms/step, estimated remaining run time = 0.02 min)
Pushed 160000/200000 steps (80.00%, 0.000022 ms/step, estimated remaining run time = 0.01 min)
Pushed 180000/200000 steps (90.00%, 0.000021 ms/step, estimated remaining run time = 0.01 min)
Done!


In [2]:
sim.name = "fig_setup_IC_array"
sim.save_data()

In [7]:
rng = np.random.default_rng()

W_para_min = Quantity(50, "eV")
W_para_min = SimQuantity(W_factor.user_to_code(W_para_min), W_para_min)
V_para_min = energy_to_speed(W_para_min, units)

W_perp_min = Quantity(0.0, "eV")
W_perp_min = SimQuantity(W_factor.user_to_code(W_perp_min), W_perp_min)
V_perp_min = energy_to_speed(W_perp_min, units)

W_max = Quantity(2000, "eV")
W_max = SimQuantity(W_factor.user_to_code(W_max), W_max)
V_max = energy_to_speed(W_max, units)

Np = 50_000
V_para = rng.uniform(V_para_min.code, V_max.code, Np)
V_perp = rng.uniform(V_perp_min.code, V_max.code, Np)
phase = rng.uniform(-np.pi, np.pi, Np)
uz0 = V_para
ux0 = V_perp * np.cos(phase)
uy0 = V_perp * np.sin(phase)
x0 = np.zeros_like(uz0)
y0 = np.zeros_like(uz0)
z0 = np.zeros_like(uz0) - 3.0 * R
g0 = lorentz_factor(ux0, uy0, uz0, c)
ICs = Particle(units.electron, 0.0, g0, x0, y0, z0, ux0, uy0, uz0)

sim.run(
    initial_conditions=ICs,
    run_time=500.0 * Tc,
    step_size=1e-2 * Tc,
    save_intervals=5,
    log_intervals=100,
)

Pushed 0/50000 steps (0.00%, 0.000025 ms/step, estimated remaining run time = 0.02 min)
Pushed 500/50000 steps (1.00%, 0.010491 ms/step, estimated remaining run time = 8.66 min)
Pushed 1000/50000 steps (2.00%, 0.009673 ms/step, estimated remaining run time = 7.91 min)
Pushed 1500/50000 steps (3.00%, 0.009751 ms/step, estimated remaining run time = 7.89 min)
Pushed 2000/50000 steps (4.00%, 0.009990 ms/step, estimated remaining run time = 8.00 min)
Pushed 2500/50000 steps (5.00%, 0.009926 ms/step, estimated remaining run time = 7.86 min)
Pushed 3000/50000 steps (6.00%, 0.010200 ms/step, estimated remaining run time = 8.00 min)
Pushed 3500/50000 steps (7.00%, 0.009809 ms/step, estimated remaining run time = 7.61 min)
Pushed 4000/50000 steps (8.00%, 0.010174 ms/step, estimated remaining run time = 7.81 min)
Pushed 4500/50000 steps (9.00%, 0.009835 ms/step, estimated remaining run time = 7.46 min)
Pushed 5000/50000 steps (10.00%, 0.010180 ms/step, estimated remaining run time = 7.64 min)
Pu

In [8]:
sim.name = "fig_setup_IC_random"
sim.save_data()